# GTFS

## Create a geopandas geodataframe from a GTFS feed

Here is a url for a GTFS data feed. Let's turn it into a flexible geodataframe!

In [ ]:
url = 'http://web.mta.info/developers/data/nyct/subway/google_transit.zip'

We first have to import some modules from the [Python standard library](https://docs.python.org/3/library/)

We also have to import some third party modules.

Add the *conda-forge* channel to your base channel by running:

`conda config --add channels conda-forge`

You can then create an environment with these dependencies by running: 

`conda create --name geo_env --file package-list.txt`

In [ ]:
import requests
from zipfile import ZipFile
from io import StringIO, BytesIO

import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import contextily as ctx
from shapely import geometry

Define a function for reading gtfs zipfiles into python dataframes. The dataframes are stored in a python dictionary.

In [ ]:
def zipToDataframes(zip):
    dataframes = {}
    for file in zip.filelist:
        file_name = file.filename
        with zip.open(file_name) as f:
            bytes = f.read()
            s = str(bytes, 'utf-8')
            data = StringIO(s)
            df = pd.read_csv(data, low_memory=False)
            name = file_name.split('.txt')[0]
            dataframes[name] = df
    return dataframes

Run the function and list the resulting dataframes. The number of dataframes will vary between different gtfs sources.

*You can also work with a local copy of the data for improved performance*

In [ ]:
#zip = zipfile.ZipFile('data/nyc_subways.zip')
r = requests.get(url)
zip = ZipFile(BytesIO(r.content))
gtfs_dataframes = zipToDataframes(zip)
list(gtfs_dataframes)

Let's take a look at the agency dataframe

In [ ]:
agency = gtfs_dataframes['agency']
agency.head()

Here's the routes dataframe

In [ ]:
routes = gtfs_dataframes['routes']
routes.head()

We can join the agency and routes dataframes on the agency_id column

In [ ]:
agency_routes = agency.join(
    routes.set_index('agency_id'),
    on='agency_id'
)
agency_routes.head()

Here's a look at the trips datframe

In [ ]:
trips = gtfs_dataframes['trips']
trips.head()

We can join the trips and routes dataframes on the route_id

In [ ]:
routes_trips = agency_routes.join(
    trips.set_index('route_id'),
    on='route_id'
)
routes_trips.head()

This is the shapes dataframe. It contains the spatial data for the trips in the GTFS feed.

In [ ]:
shapes = gtfs_dataframes['shapes']
shapes.head()

We can join the shapes and trips on the shape_id column. Let's also drop a bunch of columns that no longer need.

In [ ]:
shapes_trips = shapes.join(
    routes_trips.set_index('shape_id'),
    on='shape_id'
)

shapes_trips.drop(
    [
        'service_id',
        'trip_id',
        'trip_headsign',
        'direction_id',
        'block_id',
        'shape_pt_sequence',
        'shape_dist_traveled'
    ], 
    axis=1
).reset_index(drop=True)

Let's list the unique route ids

In [ ]:
list(routes.route_id.unique())

Now for the fun part... We're going to transform this data into a geodataframe. Additional information describing each step is provided in the code comments.

In [ ]:
# Create a list to store the shapes for each route
route_list = []

# For each unique route_id
for route_id in routes.route_id.unique():
    
    # get the route shapes
    route_shapes = shapes_trips.loc[shapes_trips.route_id == route_id]
    
    # check if there are shapes
    if route_shapes.size > 0:
        
        # get the points for each shape
        route_shapes = route_shapes.drop_duplicates(
            subset=['shape_id', 'shape_pt_lat', 'shape_pt_lon'],
            keep='first'
        ).reset_index(drop=True)

        # add a shapely geometry column
        route_shapes['geometry'] = route_shapes.apply(
            lambda row: geometry.Point(row.shape_pt_lon, row.shape_pt_lat),
            axis=1
        )

        # create a new dataframe with one row for each route shape
        unique_route_shapes = route_shapes.drop_duplicates(
            subset=['shape_id'],
            keep='first'
        ).drop(
            [
                'shape_pt_lat',
                'shape_pt_lon'
            ], 
            axis=1
        ).reset_index(drop=True)

        # create a linestring for each shape from its points
        unique_route_shapes['geometry'] = unique_route_shapes.apply(
            lambda row: geometry.LineString(
                route_shapes.loc[route_shapes.shape_id == row.shape_id].geometry.to_list()
            ),
            axis=1
        )

        # append the shapes to the route list
        route_list.append(unique_route_shapes)

# create a geodataframe from the route list
network = gpd.GeoDataFrame(pd.concat(route_list)).reset_index(drop=True)
network.set_crs(4236, inplace=True) # set the spatial reference
network.to_crs(epsg=3857, inplace=True) # project the coordinates
network.head()

Check if there is a route color and set a default, if none.

In [ ]:
default_color = '000000'
if 'route_color' in network.columns:
    network.route_color.fillna(default_color, inplace=True)
else:
    network.route_color = default_color

Now we can plot the geodataframe

In [ ]:
ax = network.plot(color='#' + network.route_color, figsize=(10, 10), alpha=0.5)
ctx.add_basemap(ax)
plt.show()

Contextily provides a number of basemap sources

In [ ]:
list(ctx.providers.keys())

Let's take a look at CartoDB

In [ ]:
list(ctx.providers.CartoDB.keys())

Here's our final plot of the GTFS routes!

In [ ]:
ax = network.plot(color='#' + network.route_color, figsize=(10, 10), alpha=0.5)
ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)
plt.show()